In [1]:
import argparse
import numpy as np
import torch
from espnet2.bin.tts_inference import Text2Speech
from espnet2.utils.types import str_or_none

In [3]:
output_fname = "output.wav"

fs, lang = 44100, "Japanese"
model='exp/tts_train_raw_phn_jaconv_pyopenjtalk_prosody/178epoch.pth'

text2speech = Text2Speech.from_pretrained(
    model_file=model,
    device="cpu"
)

In [4]:
pause = np.zeros(30000, dtype=np.float32)

x = "私の名前は岩崎です"
sentence_list = x.split('<pause>')

wav_list = []

In [5]:
for sentence in sentence_list:
    with torch.no_grad():
        result = text2speech(sentence)["wav"]
        wav_list.append(np.concatenate([result.view(-1).cpu().numpy(), pause]))

final_wav = np.concatenate(wav_list)

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


In [12]:
result = text2speech(sentence)

In [17]:
type(result)
print(result.keys())

dict_keys(['feat_gen', 'prob', 'att_w', 'feat_gen_denorm', 'duration', 'focus_rate', 'wav'])


In [6]:
rom scipy.io.wavfile import write
write(output_fname, rate=text2speech.fs, data=final_wav)

SyntaxError: invalid syntax (3523856632.py, line 1)

In [18]:
import pyopenjtalk

# phones is a str object separated by space
phones = pyopenjtalk.g2p('こんにちは', kana=False)
phones = phones.split(" ")
print(phones)

['k', 'o', 'N', 'n', 'i', 'ch', 'i', 'w', 'a']


In [19]:
import re

phones = []
for labels in pyopenjtalk.run_frontend("こんにちは")[1]:
    p = re.findall(r"\-(.*?)\+.*?\/A:([0-9\-]+).*?\/F:.*?_([0-9]+)", labels)
    if len(p) == 1:
        phones += [p[0][0], p[0][2], p[0][1]]

print(phones)

['k', '5', '-4', 'o', '5', '-4', 'N', '5', '-3', 'n', '5', '-2', 'i', '5', '-2', 'ch', '5', '-1', 'i', '5', '-1', 'w', '5', '0', 'a', '5', '0']


In [2]:
from espnet2.text.phoneme_tokenizer import PhonemeTokenizer
phone = PhonemeTokenizer(g2p_type = "pyopenjtalk_prosody")

In [5]:
result = phone.text2tokens("私の名前は岩崎です")
print(type(result))

<class 'list'>
